In [60]:
import pdfplumber
import os
import json

from openai import OpenAI




In [61]:
with pdfplumber.open('data/business_license_data.pdf') as pdf:
    reversed_lines = []
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines = text.split('\n')
            reversed_lines.extend([line[::-1] for line in lines])

all_text_pdf = '\n'.join(reversed_lines)

lines = all_text_pdf.split('\n')
requirements_list = []
for line in lines:
    if any(word in line for word in ["דרישה", "חובה", "על העסק", "נדרש"]):
        requirements_list.append(line.strip())

os.makedirs('output', exist_ok=True)

with open('output/requirements.json', 'w', encoding='utf-8') as f:
    json.dump({"requirements": requirements_list}, f, ensure_ascii=False, indent=4)

print("✅ Created 'output/requirements.json'")

client = OpenAI()
rules = []

for req in requirements_list:
    print(f"🔍 Processing requirement: {req}")
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "אתה מקטלג דרישות רגולטוריות לפי סוג העסק."
            },
            {
                "role": "user",
                "content": f"הדרישה היא: \"{req}\".\nתגיד לי איזה תנאים (type וכו') מתאימים לעסק שצריך לקיים את הדרישה הזאת. תשיב במבנה JSON עם 'conditions' ו-'requirement'."
            }
        ]
    )
    answer = response.choices[0].message.content
    rule = json.loads(answer)
    rules.append(rule)

with open('output/rules.json', 'w', encoding='utf-8') as f:
    json.dump(rules, f, ensure_ascii=False, indent=4)

print("✅ Created 'output/rules.json'")


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\eelis\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
